In [30]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
import os

load_dotenv()

uri = os.getenv("MONGODB_URI")
client = MongoClient(uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [6]:
from langchain.prompts import ChatPromptTemplate 
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.document_loaders import WebBaseLoader
from youtube_transcript_api import YouTubeTranscriptApi
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, Runnable
import bs4
from operator import itemgetter
import markdown
import datetime
import pytube
import os

async def web_loader(url: str):
    loader = WebBaseLoader(web_paths=[url])
    docs = []
    async for doc in loader.alazy_load():
        docs.append(doc)
    return docs

def chain_summarization(content: str, size: str = 'medium'):
    
    if size == 'small':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar textos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de uma página web e gerar uma sumarização curta que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}')] 
    elif size == 'medium':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar textos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de uma página web e gerar uma sumarização de tamanho médio que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}')]
    elif size == 'large':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar textos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de uma página web e gerar uma sumarização longa e detalhada que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}.')]

    prompt = ChatPromptTemplate.from_messages(messages)
    llm = ChatOpenAI(model='gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))
    chain = (
        {
            'texto': itemgetter('texto')
        }
        |prompt
        | llm
        |StrOutputParser()
    )

    resume = chain.invoke({'texto': content})
    return resume

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
'''
Schema pra guardar:
- url
- content_type (web, youtube)
- original_content (clipado [:3_000])
- summarization 
- who_added
'''

'\nSchema pra guardar:\n- url\n- content_type (web, youtube)\n- original_content (clipado [:3_000])\n- summarization \n'

In [52]:
from datetime import datetime

In [42]:
url = 'https://www.uol.com.br/'
content_type = 'web'
original_content = await web_loader(url)
original_content_str = "\n".join([x.page_content for x in original_content])[:3_000]
summarization = chain_summarization(original_content_str)
who_added = 'xvalentim'

In [43]:
load_dotenv()

DB_NAME = os.getenv('DB_NAME')
COLLECTION_NAME = os.getenv('COLLECTION_NAME')
URI = os.getenv('MONGODB_URI')

In [44]:
collection = client[DB_NAME][COLLECTION_NAME]

In [45]:
item_details = collection.find()
for item in item_details:
   # This does not give a very readable output
   print(item)

In [ ]:
collection.insert_one({
    'url': url,
    'content_type': content_type,
    'original_content': original_content_str,
    'summarization': summarization,
    'who_added': who_added,
    'date_added': datetime.datetime.now()
})

InsertOneResult(ObjectId('673b7dbc22ef674d019cee04'), acknowledged=True)

In [49]:
item_details = collection.find()
for item in item_details:
   # This does not give a very readable output
   print(item)

{'_id': ObjectId('673b7dbc22ef674d019cee04'), 'url': 'https://www.uol.com.br/', 'content_type': 'web', 'original_content': 'UOL - Seu universo online\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Ingresso.com\n        \n          Bate-Papo\n        \n          UOL HOST\n        \n          Passei Direto\n        \n          UOL Play\n        \n          PagBank\n        \n          UOL ADS\n        \n          Sac\n        \n          Email\n        \n          ASSINE UOL\n        \n      UOL - Seu universo online\n    PesquisarSiteProdutosSiteConteúdo ExclusivoSiteUOL MailSiteUOL PlaySiteUFCSiteChampions LeagueSiteNBASiteBate-Papo UOLSiteUOL Leia+SiteUOL AntivírusSiteUOL Assistência TécnicaSiteUOL ResolveSiteUOL WI-FISiteUOL SexoSiteClube UOLSiteUOL adsSiteUOL HostSiteUOL Meu NegócioSiteUOL Afiliados\n    Assine UOL\n  \n    Tenha acesso ilimitad

In [50]:
collection.delete_one({'url': url})

DeleteResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000010'), 'opTime': {'ts': Timestamp(1731952578, 4), 't': 16}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1731952578, 4), 'signature': {'hash': b'\xddj\xb2\xfd\xec\xf8\xbf(R\x18\xd3\x94\xe4U\xa6^\x94e\xab\xe1', 'keyId': 7406339075210215426}}, 'operationTime': Timestamp(1731952578, 4)}, acknowledged=True)

In [53]:
item_details = collection.find()
for item in item_details:
   # This does not give a very readable output
   print(item)

In [143]:
import datetime

def check_if_url_exists(url: str):
    item_details = collection.find({'url': url})
    for item in item_details:
        return True
    return False

def get_only_window_old_items(window: int, granularity: str):
    if granularity == 'hours':
        item_details = collection.find({'date_added': {'$gte': datetime.datetime.now() - datetime.timedelta(hours=window)}})
    elif granularity == 'days':
        item_details = collection.find({'date_added': {'$gte': datetime.datetime.now() - datetime.timedelta(days=window)}})
    elif granularity == 'minutes':
        item_details = collection.find({'date_added': {'$gte': datetime.datetime.now() - datetime.timedelta(minutes=window)}})
    else:
        return None
    items = []
    for item in item_details:
        items.append(item)
    return items

def format_object(url, content_type, original_content, summarization, who_added, date_added):
    return {
        'url': url,
        'content_type': content_type,
        'original_content': original_content,
        'summarization': summarization,
        'who_added': who_added,
        'date_added': date_added
    }
    
def count_items_per_user(username: str, window_time: int=10, granularity: str='minutes'):
    if granularity == 'hours':
        item_details = collection.find({'who_added': username, 
                                    'date_added': {'$gte': datetime.datetime.now() - datetime.timedelta(hours=window_time)}})
    elif granularity == 'days':
        item_details = collection.find({'who_added': username, 
                                    'date_added': {'$gte': datetime.datetime.now() - datetime.timedelta(days=10)}})
    elif granularity == 'minutes':
        item_details = collection.find({'who_added': username, 
                                    'date_added': {'$gte': datetime.datetime.now() - datetime.timedelta(minutes=window_time)}})
    count = 0
    for item in item_details:
        count += 1
    return count

async def get_metadata(url: str, content_type: str, username: str):
    
    if content_type == 'web':
        original_content = await web_loader(url)
        original_content_str = "\n".join([x.page_content for x in original_content])[:3_000]
        summarization = chain_summarization(original_content_str)
        return format_object(url, content_type, original_content_str, summarization, username, datetime.datetime.now())
    elif content_type == 'youtube':
        return None
    else:
        return None

def user_can_add_content(username: str, window_time: int=10, granularity: str='minutes', limit: int=5):
    return count_items_per_user(username, window_time, granularity) < limit
    
def insert_objects(objects, window_time: int=10, granularity: str='minutes'):
    for obj in objects:
        username = obj['who_added']
        url_exists = check_if_url_exists(obj['url'])
        user_can_add = user_can_add_content(username)
        if not url_exists and user_can_add:
            collection.insert_one(obj)
        else:
            if not user_can_add:
                print('User cannot add more content')
            if url_exists:
                print('URL already exists')
                

def format_text(x: dict):
    return f'URL: {x["url"]}\nTipo de conteúdo: {x["content_type"]}\nQuem adicionou: {x["who_added"]}\nData de adição: {x["date_added"]}\nConteudo resumido: {x["summarization"]}\n'                

def create_newsletter():
    all_content = "\n".join([format_text(x) for x in get_only_window_old_items(window=1, granularity='days')])
    
    messages = [
        ('system', 'Você é um assistente virtual que vai receber uma lista de conteúdos adicionados por alunos para que uma newsletter fosse criada. Sua tarefa é organizar esses conteúdos em uma newsletter, destacando os principais pontos de cada conteúdo e explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a newsletter, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), 
        ('user', 'aqui está o conteudo {all_content}')] 
    prompt = ChatPromptTemplate.from_messages(messages)
    llm = ChatOpenAI(model='gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))
    chain = (
        {
            'all_content': itemgetter('all_content')
        }
        | prompt
        | llm
        | StrOutputParser()
    )
    return chain.invoke({'all_content': all_content})

In [137]:
[format_text(x) for x in get_only_window_old_items(window=1, granularity='days')]

['URL: https://www.uol.com.br/\nTipo de conteúdo: web\nQuem adicionou: xvalentim\nData de adição: 2024-11-18 15:40:22.164000\nConteudo resumido: O conteúdo apresentado é uma visão geral do portal UOL, que oferece uma ampla gama de serviços e informações. Aqui estão os principais pontos destacados:\n\n1. **Serviços e Produtos**: O UOL disponibiliza diversas ferramentas e serviços, incluindo UOL Play (streaming de vídeos), PagBank (serviços bancários), UOL Host (hospedagem de sites), entre outros. Isso permite que os usuários acessem conteúdos e serviços variados em um único lugar.\n\n2. **Assinaturas**: Há uma opção de assinatura que oferece acesso ilimitado a conteúdos exclusivos e produtos do UOL. Os usuários podem se inscrever facilmente pelo site ou por meio de contato telefônico.\n\n3. **Conteúdo Diversificado**: O portal abrange uma vasta gama de temas, como notícias, esportes, cultura, economia, e entretenimento. Isso significa que os visitantes podem encontrar informações sobre 

In [135]:
get_only_window_old_items(window=1, granularity='days')

[{'_id': ObjectId('673b8a8a22ef674d019cee05'),
  'url': 'https://www.uol.com.br/',
  'content_type': 'web',
  'original_content': 'UOL - Seu universo online\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Ingresso.com\n        \n          Bate-Papo\n        \n          UOL HOST\n        \n          Passei Direto\n        \n          UOL Play\n        \n          PagBank\n        \n          UOL ADS\n        \n          Sac\n        \n          Email\n        \n          ASSINE UOL\n        \n      UOL - Seu universo online\n    PesquisarSiteProdutosSiteConteúdo ExclusivoSiteUOL MailSiteUOL PlaySiteUFCSiteChampions LeagueSiteNBASiteBate-Papo UOLSiteUOL Leia+SiteUOL AntivírusSiteUOL Assistência TécnicaSiteUOL ResolveSiteUOL WI-FISiteUOL SexoSiteClube UOLSiteUOL adsSiteUOL HostSiteUOL Meu NegócioSiteUOL Afiliados\n    Assine UOL\n  \n    Tenha acesso i

In [144]:
newsletter = create_newsletter()

In [147]:
print(len(newsletter))

4618
